In [1]:
import os
import time
import pandas as pd
import openai
import re
import requests
import sys
from num2words import num2words
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

openai.api_key = os.getenv("OPENAI_API_KEY") 
openai.organization = os.getenv("OPENAI_ORGANIZATION") 

start_time=time.time()
path ='c:\\temp_storage\\articles\\cognitive-services\\'

########### This helps takes care of removing metadata
search_string = "---" 
metadata_counter = 0
############
d = []
text=""

for root, directories, files in os.walk(path , topdown=False):
    for file in files:
        if file.lower().endswith(".md"):
            name =(os.path.join(root,file))
            f = open(name, "r",encoding="utf-8")
            for line in f:
                if line.find(search_string) !=-1 and metadata_counter !=2:
                    metadata_counter+=1
                if line.find(search_string) != 0 and metadata_counter==2:
                    text +=line
            f.close()
            d.append({'FILE NAME': file ,'CONTENT': text})
            pd.DataFrame(d)
            metadata_counter = 0
            text=""
end_time = time.time()
duration = end_time - start_time

print ("Script Execution: ", duration)

Script Execution:  1.3523163795471191


In [2]:
df = pd.DataFrame(d)
df

,FILE NAME,CONTENT
0,anomaly-detection-best-practices.md,\n# Best practices for using the Anomaly Detec...
1,best-practices-multivariate.md,\n# Best practices for using the Multivariate ...
2,multivariate-architecture.md,\n# Predictive maintenance solution with Multi...
3,troubleshoot.md,\n# Troubleshoot the multivariate API\n\nThis ...
4,batch-inference.md,\n# Trigger batch inference with trained model...
...,...,...
1837,rotate-keys.md,\n# Rotate subscription keys in Cognitive Serv...
1838,security-controls-policy.md,# Azure Policy Regulatory Compliance controls ...
1839,security-features.md,\n# Azure Cognitive Services security\n\nSecur...
1840,use-key-vault.md,\n# Develop Azure Cognitive Services applicati...


In [3]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.replace("#","")
    s = s.strip()
    if s =="":
        s = "<blank>"
    return s
df_normalized=df.copy()
df_normalized['CONTENT'] = df["CONTENT"].apply(lambda x : normalize_text(x))

In [4]:
df_normalized


,FILE NAME,CONTENT
0,anomaly-detection-best-practices.md,Best practices for using the Anomaly Detector ...
1,best-practices-multivariate.md,Best practices for using the Multivariate Anom...
2,multivariate-architecture.md,Predictive maintenance solution with Multivari...
3,troubleshoot.md,Troubleshoot the multivariate API This article...
4,batch-inference.md,Trigger batch inference with trained model You...
...,...,...
1837,rotate-keys.md,Rotate subscription keys in Cognitive Services...
1838,security-controls-policy.md,Azure Policy Regulatory Compliance controls fo...
1839,security-features.md,Azure Cognitive Services security Security sho...
1840,use-key-vault.md,Develop Azure Cognitive Services applications ...


In [5]:
df_blank = df_normalized[df_normalized.CONTENT=='<blank>']
print(df_blank)

                                              FILE NAME  CONTENT
57    cognitive-services-bing-custom-search-signup-r...  <blank>
61                                    publish-revert.md  <blank>
1089                  quickstart-sdk-cognitive-model.md  <blank>
1099                                  tip-debug-json.md  <blank>
1526                              quickstart-list-go.md  <blank>
1527                                 quickstart-list.md  <blank>


In [6]:
df_normalized = df_normalized[df_normalized.CONTENT !='<blank>']
len(df_normalized)
df_normalized

,FILE NAME,CONTENT
0,anomaly-detection-best-practices.md,Best practices for using the Anomaly Detector ...
1,best-practices-multivariate.md,Best practices for using the Multivariate Anom...
2,multivariate-architecture.md,Predictive maintenance solution with Multivari...
3,troubleshoot.md,Troubleshoot the multivariate API This article...
4,batch-inference.md,Trigger batch inference with trained model You...
...,...,...
1837,rotate-keys.md,Rotate subscription keys in Cognitive Services...
1838,security-controls-policy.md,Azure Policy Regulatory Compliance controls fo...
1839,security-features.md,Azure Cognitive Services security Security sho...
1840,use-key-vault.md,Develop Azure Cognitive Services applications ...


In [7]:
df_normalized

,FILE NAME,CONTENT
0,anomaly-detection-best-practices.md,Best practices for using the Anomaly Detector ...
1,best-practices-multivariate.md,Best practices for using the Multivariate Anom...
2,multivariate-architecture.md,Predictive maintenance solution with Multivari...
3,troubleshoot.md,Troubleshoot the multivariate API This article...
4,batch-inference.md,Trigger batch inference with trained model You...
...,...,...
1837,rotate-keys.md,Rotate subscription keys in Cognitive Services...
1838,security-controls-policy.md,Azure Policy Regulatory Compliance controls fo...
1839,security-features.md,Azure Cognitive Services security Security sho...
1840,use-key-vault.md,Develop Azure Cognitive Services applications ...


| GENERATION |TOKENIZER    | MAX INPUT TOKENS| KNOWLEDGE CUTOFF|
|------------|-------------|-----------------|-----------------|
| V2         | cl100k_base | 8191            | Sep 2021        |
| V1         | GPT-2/GPT-3 | 2046            | Aug 2020        |


https://beta.openai.com/docs/guides/embeddings/what-are-embeddings

https://openai.com/blog/new-and-improved-embedding-model/

In [8]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_tok=df_normalized.copy()
df_tok['n_tokens'] = df_normalized["CONTENT"].apply(lambda x: len(tokenizer.encode(x)))
df_tok

,FILE NAME,CONTENT,n_tokens
0,anomaly-detection-best-practices.md,Best practices for using the Anomaly Detector ...,1244
1,best-practices-multivariate.md,Best practices for using the Multivariate Anom...,3318
2,multivariate-architecture.md,Predictive maintenance solution with Multivari...,789
3,troubleshoot.md,Troubleshoot the multivariate API This article...,2196
4,batch-inference.md,Trigger batch inference with trained model You...,2638
...,...,...,...
1837,rotate-keys.md,Rotate subscription keys in Cognitive Services...,310
1838,security-controls-policy.md,Azure Policy Regulatory Compliance controls fo...,215
1839,security-features.md,Azure Cognitive Services security Security sho...,1551
1840,use-key-vault.md,Develop Azure Cognitive Services applications ...,4827


In [9]:

# Based on https://openai.com/api/pricing/ on 01/29/2023
# If you were using this for approximating pricing with Azure OpenAI adjust the values below with: https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/

#MODEL	USAGE
#Ada     v1	$0.0040 / 1K tokens
#Babbage v1	$0.0050 / 1K tokens
#Curie   v1	$0.0200 / 1K tokens
#Davinci v1	$0.2000 / 1K tokens

#MODEL	USAGE
#Ada     v2	$0.0004 / 1K tokens
#This Ada model, text-embedding-ada-002, is a better and lower cost replacement for our older embedding models. 

n_tokens_sum = df_tok['n_tokens'].sum()

ada_v1_embeddings_cost = (n_tokens_sum/1000) *.0040
babbage_v1_embeddings_cost = (n_tokens_sum/1000) *.0050
curie_v1_embeddings_cost = (n_tokens_sum/1000) *.02
davinci_v1_embeddings_cost = (n_tokens_sum/1000) *.2

ada_v2_embeddings_cost = (n_tokens_sum/1000) *.0004

print("Number of tokens: " + str(n_tokens_sum) + "\n")

print("MODEL        VERSION    COST")
print("-----------------------------------")
print("Ada" + "\t\t" + "v1" + "\t$" + '%.8s' % str(ada_v1_embeddings_cost))
print("Babbage" + "\t\t" + "v1" + "\t$" + '%.8s' % str(babbage_v1_embeddings_cost))
print("Curie" + "\t\t" + "v1" + "\t$" + '%.8s' % str(curie_v1_embeddings_cost))
print("Davinci" + "\t\t" + "v1" + "\t$" + '%.8s' % str(davinci_v1_embeddings_cost))
print("Ada" + "\t\t" + "v2" + "\t$" + '%.8s' %str(ada_v2_embeddings_cost))

Number of tokens: 2532696

MODEL        VERSION    COST
-----------------------------------
Ada		v1	$10.13078
Babbage		v1	$12.66348
Curie		v1	$50.65392
Davinci		v1	$506.5392
Ada		v2	$1.013078


In [10]:
len(df)

1842

In [11]:
df_tok = df_tok[df_tok.n_tokens<400]

len(df_tok)

534

# OpenAI Text & Embedding Rate Limits?

Rate limits are enforced at the **organization level, not user level**, based on the specific endpoint used as well as the type of account you have. 

Rate limits are measured in two ways: **RPM (requests per minute)** and **TPM (tokens per minute)**. 

## TEXT & EMBEDDING

Free trial users •20 RPM •150,000 TPM

Pay-as-you-go users (first 48 hours)	•60 RPM •250,000 TPM

Pay-as-you-go users (after 48 hours)	•3,000 RPM •250,000 TPM

https://beta.openai.com/docs/guides/rate-limits/overview

In [ ]:
import time
from IPython.display import clear_output

request_counter = 0
total_requests_sent = 0
rate_limit= 3000

start_timer=time.time()

def generate_embeddings(text, model="text-embedding-ada-002"):
    global request_counter
    global rate_limit
    global total_requests_sent
    global start_timer
    clear_output(wait=True)
    check_timer = time.time()
    duration = check_timer-start_timer
    print(duration)
    
    if int(duration) >= 60:
        start_timer=time.time()
        request_counter=0
    if request_counter == rate_limit and int(duration) <= 59:
        sleep_for = 60-int(duration)
        print("Sleeping for " + str(sleep_for) +" seconds")
        print("Total requests sent: ", total_requests_sent)
        time.sleep(sleep_for)
        start_timer = time.time()
        request_counter =0
    if request_counter < rate_limit:
        request_counter+=1
        total_requests_sent+=1
        print("Request counter: ", request_counter)
        print("Total requests sent: ", total_requests_sent)
        
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

df_embeddings = df_tok.copy()
df_embeddings['ada_v2_embedding'] = df_tok.CONTENT.transform(lambda x: generate_embeddings(x, model='text-embedding-ada-002'))

df_embeddings


In [51]:
# search embedded docs based on cosine similarity

df_similarities = df_embeddings.copy()

def get_embedding(text, model="text-embedding-ada-002"):
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002"
    )

    df_similarities["similarities"] = df_embeddings.ada_v2_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df_similarities.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

question = input("How can I help you?\n\n")

res = search_docs(df, question, top_n=3)

How can I help you?

 test


,FILE NAME,CONTENT,n_tokens,ada_v2_embedding,similarities
713,test-model.md,To test your model from Language Studio 1. Sel...,165,"[0.0032791579142212868, -0.0011860255617648363...",0.780951
874,do-not-use-too-few-utterances.md,These few utterances are for demonstration pur...,53,"[-0.0060370368883013725, 0.005788491573184729,...",0.772646
867,best-practice-utterances.md,For best results make sure that example uttera...,54,"[-0.014310219325125217, 0.03167257085442543, 0...",0.770794


In [49]:
res.CONTENT.values[0]

'To test your model from Language Studio 1. Select **Testing deployments** from the left side menu. 3. Select the model you want to test. You can only test models that are assigned to deployments. 4. From deployment name dropdown, select your deployment name. 5. In the text box, enter an utterance to test. 6. From the top menu, click on **Run the test**. 7. After you run the test, you should see the response of the model in the result. You can view the results in entities cards view, or view it in JSON format. :::image type="content" source="././media/test-model.png" alt-text="A screenshot showing how to test a model in Language Studio." lightbox="././media/test-model.png":::'

In [52]:
df_similarities

,FILE NAME,CONTENT,n_tokens,ada_v2_embedding,similarities
25,quickstart-cleanup-next-steps.md,Clean up resources If you want to clean up and...,188,"[-0.002563188085332513, 0.005606868304312229, ...",0.706685
26,client-libraries-multivariate.md,Quickstart: Use the Multivariate Anomaly Detec...,150,"[-0.019996939226984978, 0.011973044835031033, ...",0.729733
27,client-libraries.md,Quickstart: Use the Univariate Anomaly Detecto...,147,"[-0.003396739950403571, 0.013678478077054024, ...",0.727968
46,client-libraries.md,Quickstart: Use the Bing Autosuggest client li...,101,"[-0.014453585259616375, 0.017982948571443558, ...",0.711676
62,client-libraries.md,Quickstart: Use the Bing Custom Search client ...,130,"[-0.000941423000767827, 0.024544676765799522, ...",0.719405
...,...,...,...,...,...
1767,prerequisites-go.md,Prerequisites This quickstart requires: * [Go]...,39,"[0.0007572504691779613, 0.0019182561663910747,...",0.734317
1768,prerequisites-java.md,Prerequisites This quickstart requires: * [JDK...,64,"[0.010907812044024467, -0.005134829320013523, ...",0.737251
1769,prerequisites-nodejs.md,Prerequisites This quickstart requires: * [Nod...,46,"[0.0030772036407142878, 4.518704008660279e-05,...",0.726393
1770,prerequisites-python.md,Prerequisites This quickstart requires: * [Pyt...,48,"[0.005925570148974657, 0.0006811613566242158, ...",0.729803


In [54]:
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002"
    )
    
    df_question = df_similarities.copy()
    df_question["similarities"] = df_similarities.ada_v2_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df_question.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    return res

res = search_docs(df, question, top_n=1)

ai_question = input("How can I help you?\n\n")
context= res.CONTENT.values
completion_model='text-davinci-003'

initial_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly."

combined_prompt = initial_prompt + str(context) + "Q: " + ai_question
response = openai.Completion.create(model=completion_model, prompt=combined_prompt, max_tokens=100)
ai_response = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()

print("\n"+ ai_response)

How can I help you?

 Tell me about Multivariate Anomaly Detection?



Multivariate Anomaly Detection (MAD) is a type of anomaly detection technology that uses machine learning to detect anomalies across multiple datasets. It is used to uncover hidden relationships between variables, to detect outliers and anomalies, and to assess the degree of change. MAD can help identify potential problems in areas such as credit card fraud and financial fraud, or detect patterns in industrial process control systems.
